# Importações

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from math import sqrt

from sklearn.tree import DecisionTreeClassifier

from statistics import mean 

# Leitura de dados e labels

In [0]:
data = pd.read_csv('/content/drive/My Drive/Adaboost/sonar.all-data.csv', header=None)

sonar_df = pd.DataFrame(data)
labels_df = pd.DataFrame()

# Muda a ordem das linhas do dataframe de forma aleatória
sonar_df = sonar_df.sample(frac=1)

# Valores do dataset
data = sonar_df.iloc[:, :-1].values
# Classes
labels = sonar_df.iloc[:, 60].values

# Mudando os labels: R -> -1 e M -> 1
for y in range(len(labels)):
    if labels[y] == 'R':
        labels[y] = -1.0
    else:
        labels[y] = 1.0
        
labels = labels.astype('float')

# Normalizando

In [18]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
data = scaler.fit_transform(data)

print(data)

np.min(data[:,0]), np.max(data[:,0])

[[0.08112094 0.06258037 0.06668857 ... 0.08695652 0.23415978 0.12009238]
 [0.13864307 0.04929276 0.11990802 ... 0.07551487 0.03305785 0.02309469]
 [0.18215339 0.24689241 0.35611038 ... 0.36842105 0.25895317 0.16628176]
 ...
 [0.02581121 0.00471496 0.08377135 ... 0.13729977 0.04407713 0.05080831]
 [0.30457227 0.23531933 0.22766097 ... 0.18535469 0.30853994 0.09930716]
 [0.47861357 0.24389198 0.271682   ... 0.36384439 0.29752066 0.16859122]]


(0.0, 1.0)

# Sub-amostragem

In [0]:
def sub_amostragem (tam, X, y, D):
    
    X_sub = pd.DataFrame(X) 
    y_sub = pd.DataFrame(y)
    D_sub = pd.DataFrame(D)

    '''Cada linha é um sample, se ela tiver um peso maior, será selecionada.
    Assim, se criará um novo conjunto de sub-amostragem.'''

    # Retorna o % de tam das primeiras linhas mais acima na ordenação, pois são as com maiores pesos.
    X_sub = X.head(tam)
    y_sub = y.head(tam)
    D_sub = D.head(tam)

    return X_sub, y_sub, D_sub

# AdaBoost

In [0]:
def Adaboost (X, y, D):
    
    X_ = pd.DataFrame(X) 
    y_ = pd.DataFrame(y)
    D_ = pd.DataFrame(D)
    
    # Modelo de aprendizagem fraca (árvore com profundidade 1 e, consequentemente, dois nodos filhos)
    clf = DecisionTreeClassifier(max_depth = 1, max_leaf_nodes = 2, random_state = 0)

    # Utilizando a função Dn
    clf.fit(X_, y_, sample_weight = np.array(D_[1].tolist()))

    # Hipótese Fn: X -> Y
    hipotese = clf.predict(X_)
    
    #print("Hipotese: \n" + str(hipotese))

    '''Duas variáveis controlam alguns laços abaixo, uma para o df (por causa dos seus índices) e outra para os vetores.''' 

    erros=[]
    i=0
    for index, row in y_.iterrows():
        erros.append(row.item() != hipotese[i]) # Monta um vetor de erros onde a hipotese é diferente do label
        i+=1

    #print("Erros: \n" + str(erros))

    # Erro da hipótese Fn
    erro_hipotese=0
    i=0
    for index, row in D_.iterrows():
        if erros[i]:
            erro_hipotese += row.item() # Faz o somatório dos pesos das posições erradas
        i+=1

    #print("Erro da hipotese: " + str(erro_hipotese))

    # Faça Bn = 1/2 * log(1-En)/En
    if erro_hipotese == 0: # Evita divisão por zero
        beta = 0
    else:
        beta = 0.5 * np.log((1. - erro_hipotese) / erro_hipotese)

    #print("Beta: " + str(beta))

    # Converte y de DataFrame para array para não dar problema no cálculo de D abaixo
    y_array = y_.to_numpy() 

    # Atualiza a distribuição Dn
    j=0
    for i in D_.index:
        D_.at[i,1] = D_.at[i,1] * np.exp(-beta * y_array[j] * hipotese[j])
        j+=1

    ''' Descomentar esse bloco para ver a acurácia na sub-amostragem

    # Guarda no vetor resultado a hipótese final do modelo
    resultado = []
    for j in hipotese:
      resultado.append(np.sign(j * beta))

    # Soma 1 ao total de acertos cada vez que o resultado da hipótese for igual ao label
    total=0
    k=0
    for index, row in y_.iterrows():
      if resultado[k] == row.item():
        total+=1
      k+=1
    
    print("\nResultado na sub-amostragem:", total/len(y_)*100, "%")
    '''
    
    return D_, beta, hipotese, clf

# Main

In [21]:
sonar_df = pd.DataFrame(data, index=np.arange(1, 209), columns=np.arange(1, 61))
labels_df = pd.DataFrame(labels, index=np.arange(1, 209), columns=[1])

# Inicialização: D1(i)=1/N para todo i 
pesos = len(labels_df)

# Peso inicial = 1/N
pesos = pd.DataFrame(np.ones(pesos) / pesos, index=np.arange(1, 209), columns=[1])

pesos_lista = []
betas_lista = []
hipoteses_lista = []
resultados_lista = []
y_preds = []
acuracia_modelos = []

X = pd.DataFrame(sonar_df) 
y = pd.DataFrame(labels_df)
D = pd.DataFrame(pesos)

T = 20
kf = KFold(n_splits = T)

# A sub-amostragem é feita com 10% das linhas do dataset original
sub_tam = int((y.size*10)/100)

fold = 1

for i in kf.split(X):

    # 70% treino e 30% teste - a opção de shuffle fica desligada para não mexer na ordem dos pesos
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=False)
    D_train = D.reindex(X_train.index) # Cria um conjunto de pesos D do mesmo tamanho de X_train

    X_sub, y_sub, D_sub = sub_amostragem(sub_tam, X_train, y_train, D_train)

    D_sub, beta, hipotese, modelo = Adaboost(X_sub, y_sub, D_sub)
    
    # Atualiza os pesos (D) nas posições da sub-amostragem (D_sub)
    D.update(D_sub)

    # Normaliza os pesos
    for j in D.index:
        D.at[j,1] = D.at[j,1] / sum(D)
    
    D.sort_values(by=1, ascending=False, inplace=True) # Reordena os pesos colocando os mais altos em primeiro
    X = X.reindex(D.index) # Reordena os dados conforme os índices dos pesos (D)
    y = y.reindex(D.index) # Reordena os labels também
    
    y_pred = modelo.predict(X_test)

    # Guarda no vetor resultado a hipótese final do modelo
    resultado = []
    for j in y_pred:
      resultado.append(np.sign(j * beta))

    # Soma 1 ao total de acertos cada vez que o resultado da hipótese for igual ao label
    total=0
    k=0
    for index, row in y_test.iterrows():
      if resultado[k] == row.item():
        total+=1
      k+=1
    
    betas_lista.append(beta)
    hipoteses_lista.append(hipotese)
    pesos_lista.append(D)
    y_preds.append(y_pred)
    resultados_lista.append(resultado)
    acuracia_modelos.append(total/len(y_test)*100)
    
    #Hipótese Final
    print("Acurácia do modelo", fold, ":", str(total/len(y_test)*100), "%")

    fold += 1

#Hipótese Final
print("\nMédia das acurácias:", mean(acuracia_modelos), "%")

Acurácia do modelo 1 : 58.730158730158735 %
Acurácia do modelo 2 : 66.66666666666666 %
Acurácia do modelo 3 : 50.79365079365079 %
Acurácia do modelo 4 : 80.95238095238095 %
Acurácia do modelo 5 : 69.84126984126983 %
Acurácia do modelo 6 : 57.14285714285714 %
Acurácia do modelo 7 : 57.14285714285714 %
Acurácia do modelo 8 : 57.14285714285714 %
Acurácia do modelo 9 : 50.79365079365079 %
Acurácia do modelo 10 : 46.03174603174603 %
Acurácia do modelo 11 : 79.36507936507937 %
Acurácia do modelo 12 : 68.25396825396825 %
Acurácia do modelo 13 : 58.730158730158735 %
Acurácia do modelo 14 : 77.77777777777779 %
Acurácia do modelo 15 : 52.38095238095239 %
Acurácia do modelo 16 : 61.904761904761905 %
Acurácia do modelo 17 : 68.25396825396825 %
Acurácia do modelo 18 : 46.03174603174603 %
Acurácia do modelo 19 : 57.14285714285714 %
Acurácia do modelo 20 : 63.49206349206349 %

Média das acurácias: 61.42857142857143 %
